 <H1> Predict Sales price using the data provided </H1>

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

In [17]:
import sys
!{sys.executable} -m pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [18]:
import xgboost as xgb

In [19]:
data = pd.read_csv(filepath_or_buffer = 'https://raw.githubusercontent.com/insaid2018/Term-4/master/Data/Housing.csv')


In [20]:
data.head()

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [21]:
data.shape

(546, 12)

In [22]:
data.describe()

,price,lotsize,bedrooms,bathrms,stories,garagepl
count,546.000000,546.000000,546.000000,546.000000,546.000000,546.000000
mean,68121.597070,5150.265568,2.965201,1.285714,1.807692,0.692308
std,26702.670926,2168.158725,0.737388,0.502158,0.868203,0.861307
min,25000.000000,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,49125.000000,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,62000.000000,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,82000.000000,6360.000000,3.000000,2.000000,2.000000,1.000000
max,190000.000000,16200.000000,6.000000,4.000000,4.000000,3.000000


Observations
1. Min. Bathrooms is One
2. Min. Bedrooms  is One
3. Min. 25% of the houses have Garage
4. Min 1 room and Max 6 rooms

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     546 non-null    float64
 1   lotsize   546 non-null    int64  
 2   bedrooms  546 non-null    int64  
 3   bathrms   546 non-null    int64  
 4   stories   546 non-null    int64  
 5   driveway  546 non-null    object 
 6   recroom   546 non-null    object 
 7   fullbase  546 non-null    object 
 8   gashw     546 non-null    object 
 9   airco     546 non-null    object 
 10  garagepl  546 non-null    int64  
 11  prefarea  546 non-null    object 
dtypes: float64(1), int64(5), object(6)
memory usage: 51.3+ KB


In [11]:
data.drop_duplicates(inplace = True)
print('Dropping Duplicates Successful')

Dropping Duplicates Successful


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     546 non-null    float64
 1   lotsize   546 non-null    int64  
 2   bedrooms  546 non-null    int64  
 3   bathrms   546 non-null    int64  
 4   stories   546 non-null    int64  
 5   driveway  546 non-null    object 
 6   recroom   546 non-null    object 
 7   fullbase  546 non-null    object 
 8   gashw     546 non-null    object 
 9   airco     546 non-null    object 
 10  garagepl  546 non-null    int64  
 11  prefarea  546 non-null    object 
dtypes: float64(1), int64(5), object(6)
memory usage: 51.3+ KB


1 duplicate column dropped

In [25]:
data = pd.get_dummies(data = data, columns = ['driveway','recroom','fullbase','gashw','airco','prefarea'])

 <h1> Output modified due to rerunning the code </h1>

In [26]:
data.head()

,price,lotsize,bedrooms,bathrms,stories,garagepl,driveway_no,driveway_yes,recroom_no,recroom_yes,fullbase_no,fullbase_yes,gashw_no,gashw_yes,airco_no,airco_yes,prefarea_no,prefarea_yes
0,42000.0,5850,3,1,2,1,False,True,True,False,False,True,True,False,True,False,True,False
1,38500.0,4000,2,1,1,0,False,True,True,False,True,False,True,False,True,False,True,False
2,49500.0,3060,3,1,1,0,False,True,True,False,True,False,True,False,True,False,True,False
3,60500.0,6650,3,1,2,0,False,True,False,True,True,False,True,False,True,False,True,False
4,61000.0,6360,2,1,1,0,False,True,True,False,True,False,True,False,True,False,True,False


In [27]:
quantile33 = data['price'].quantile(0.33)
quantile66 = data['price'].quantile(0.66)
quantile100 = data['price'].quantile(1)

data['price_cat'] = pd.cut(data['price'], bins = [0, quantile33, quantile66, quantile100], labels=['Low', 'Medium', 'High'])

data.drop(labels = 'price', axis =1, inplace = True )

print(data['price_cat']. value_counts())

data.head()

price_cat
High      185
Low       181
Medium    180
Name: count, dtype: int64


,lotsize,bedrooms,bathrms,stories,garagepl,driveway_no,driveway_yes,recroom_no,recroom_yes,fullbase_no,fullbase_yes,gashw_no,gashw_yes,airco_no,airco_yes,prefarea_no,prefarea_yes,price_cat
0,5850,3,1,2,1,False,True,True,False,False,True,True,False,True,False,True,False,Low
1,4000,2,1,1,0,False,True,True,False,True,False,True,False,True,False,True,False,Low
2,3060,3,1,1,0,False,True,True,False,True,False,True,False,True,False,True,False,Low
3,6650,3,1,2,0,False,True,False,True,True,False,True,False,True,False,True,False,Medium
4,6360,2,1,1,0,False,True,True,False,True,False,True,False,True,False,True,False,Medium


Divide data into X and Y

In [28]:
X = data.drop('price_cat', axis = 1)
y = data['price_cat']

In [29]:
print(X)
print(y)

     lotsize  bedrooms  bathrms  stories  garagepl  driveway_no  driveway_yes  \
0       5850         3        1        2         1        False          True   
1       4000         2        1        1         0        False          True   
2       3060         3        1        1         0        False          True   
3       6650         3        1        2         0        False          True   
4       6360         2        1        1         0        False          True   
..       ...       ...      ...      ...       ...          ...           ...   
541     4800         3        2        4         0        False          True   
542     6000         3        2        4         0        False          True   
543     6000         3        2        4         1        False          True   
544     6000         3        2        2         1        False          True   
545     6000         3        1        2         1        False          True   

     recroom_no  recroom_ye

In [30]:
# Split data between Test and Train
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state = 42)

# check the size of the splits
print("Train set Size: ", len(X_train))
print("Test Set Size: ", len(X_test) )

Train set Size:  382
Test Set Size:  164


In [31]:
from sklearn.preprocessing import LabelEncoder

label= LabelEncoder()

y_train= label.fit_transform(y_train)
y_test= label.fit_transform(y_test)

In [32]:
print(y_train)

[0 2 0 1 0 2 1 2 2 1 1 1 0 2 1 2 1 0 0 2 1 1 1 2 0 0 2 0 1 2 2 1 1 2 1 2 2
 2 1 1 0 1 0 1 1 1 1 1 2 1 2 2 2 2 0 0 0 0 0 2 2 0 0 2 1 0 0 1 1 0 0 1 0 2
 0 1 1 1 0 2 2 2 1 2 2 1 0 1 0 2 0 2 0 2 0 0 1 2 0 0 0 0 0 1 1 0 2 2 2 2 1
 2 0 1 1 1 1 1 0 2 1 0 0 0 1 0 2 0 2 0 0 2 2 2 1 1 0 2 2 1 0 1 2 1 1 1 0 0
 0 2 0 0 0 0 0 2 0 0 0 2 2 1 0 1 1 2 0 2 0 1 2 0 0 0 1 2 1 2 0 1 0 1 1 1 1
 0 2 0 0 0 0 2 0 0 2 2 2 2 2 0 1 0 1 1 2 1 2 2 1 2 2 1 2 1 1 1 1 0 2 1 2 1
 2 2 2 0 2 2 2 2 2 0 0 0 0 2 2 2 2 1 2 2 1 1 0 1 1 1 1 1 0 1 1 0 1 0 2 0 1
 2 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0 2 2 1 2 2 0 1 0 0 0 1 2 2 0 1 2 1 1 2 1
 0 1 0 0 2 2 2 0 2 0 0 1 0 2 2 1 1 0 0 1 2 0 2 0 1 2 2 2 1 1 0 0 0 2 2 0 2
 1 1 1 1 0 1 0 2 0 0 0 2 2 2 0 2 1 2 0 2 1 0 2 2 0 1 2 2 1 0 1 2 0 2 1 1 0
 0 1 0 2 2 0 1 1 1 2 0 0]


Initializing Base Classifiers

In [33]:
# Logistic Regression
log_clf = LogisticRegression(random_state = 42, class_weight='balanced')

# Decision Tree
dt_clf = DecisionTreeClassifier(random_state = 42, class_weight='balanced')

# Random Forest
rf_clf = RandomForestClassifier(n_estimators=500, random_state = 42, class_weight='balanced', n_jobs=-1)


In [34]:
# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=500, random_state = 42)

# Define XGBoost classifier
xgb_clf = xgb.XGBClassifier(n_estimators=500, num_class=3, random_state=42)


In [35]:
# Intialize a list of classifier objects
clf_list = [log_clf, dt_clf, rf_clf, gb_clf, xgb_clf]


In [40]:
# Create an empty list to append scores and classifier name
train_scores = []
test_scores = []
clf_names = []

# Train classifier over train data and append scores to empty list
for clf in clf_list:
  # Fit the train data over the classifier object
  clf.fit(X_train, y_train)

  # Append train and test score to the empty list
  train_scores.append(np.round(a=clf.score(X_train, y_train), decimals=2))
  test_scores.append(np.round(a=clf.score(X_test, y_test), decimals=2))
  clf_names.append(clf.__class__.__name__)

print('Success!')


Success!


In [41]:
# Create an accuracy dataframe from scores and names list
accuracy_frame = pd.DataFrame(data={'Train Accuracy': train_scores, 'Test Accuracy': test_scores}, index=clf_names)


accuracy_frame

,Train Accuracy,Test Accuracy
LogisticRegression,0.69,0.61
DecisionTreeClassifier,1.00,0.60
RandomForestClassifier,1.00,0.63
GradientBoostingClassifier,1.00,0.64
XGBClassifier,1.00,0.62


In [42]:
# Create an accuracy dataframe from scores and names list
accuracy_frame = pd.DataFrame(data={'Train Accuracy': train_scores, 'Test Accuracy': test_scores}, index=clf_names)


accuracy_frame.transpose()

,LogisticRegression,DecisionTreeClassifier,RandomForestClassifier,GradientBoostingClassifier,XGBClassifier
Train Accuracy,0.69,1.0,1.00,1.00,1.00
Test Accuracy,0.61,0.6,0.63,0.64,0.62
